In [41]:
import os
import pandas as pd
import geopandas as gpd
import json
import pathlib
import folium
from folium.plugins import MousePosition
from shapely.geometry import Point, Polygon

In [54]:
input_path = pathlib.Path('./data')
어린이교통사고_격자 = gpd.read_file(input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'),encoding="utf-8")
어린이보호구역 = pd.read_csv(input_path.joinpath('9.오산시_어린이보호구역.csv'),encoding="utf-8")
사고격자 = 어린이교통사고_격자
보호구역 = 어린이보호구역

In [55]:
# 후보 top20 불러오기
top20 = pd.read_csv('ada_top_20.csv',index_col='Unnamed: 0')
top20.head()

,gid,accident_cnt,geometry,cut,overlapped,고령인구_count_100,생산가능인구_count_100,신호등_sum,유동인구_sum,유소년인구_count_100,...,학구.raw오산초통학구역,학구.raw운산초통학구역,학구.raw운암초통학구역,학구.raw원동초통학구역,학구.raw필봉초통학구역,학구.raw화성초통학구역,정류장_count_200,총인구_count_100,pred_score,total_score
19,다사621048,10,MULTIPOLYGON (((127.0732537658753 37.141141118...,4,False,29,304,2,105.654000,13,...,0,0,0,0,0,0,7,350,5.000000,-5.000000
13,다사612101,6,MULTIPOLYGON (((127.0628451519233 37.188877953...,2,False,0,197,0,71.958583,0,...,0,0,0,0,0,0,3,207,1.603175,-4.396825
27,다사625056,6,MULTIPOLYGON (((127.077717483912 37.1483684182...,2,False,47,271,0,10.524667,24,...,0,0,0,0,0,0,1,342,1.813008,-4.186992
11,다사610057,5,MULTIPOLYGON (((127.0608214144529 37.149208429...,2,False,0,0,0,14.036517,0,...,0,0,0,0,0,0,0,0,1.558140,-3.441860
20,다사622056,4,MULTIPOLYGON (((127.0743393084319 37.148356334...,1,False,8,34,0,83.947750,0,...,0,0,1,0,0,0,3,51,1.923729,-2.076271


In [56]:
사고격자.head()

,gid,accident_cnt,geometry
0,다사551085,0,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ..."
1,다사551086,0,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ..."
2,다사551087,0,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ..."
3,다사551088,0,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ..."
4,다사552085,0,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ..."


In [57]:
# top20 gid에 대한 geometry 정보 추출 (기존 csv는 geojson이 아니라 추출 불가)

In [58]:
index=[]

for i,row in enumerate(top20.iloc):
    for j,row2 in enumerate(사고격자.iloc):
        if row.gid == row2.gid:
            index.append(j)            
target = 사고격자.iloc[index]

In [59]:
target

,gid,accident_cnt,geometry
3681,다사621048,10,"MULTIPOLYGON (((127.07325 37.14114, 127.07325 ..."
3098,다사612101,6,"MULTIPOLYGON (((127.06285 37.18888, 127.06284 ..."
3863,다사625056,6,"MULTIPOLYGON (((127.07772 37.14837, 127.07771 ..."
2902,다사610057,5,"MULTIPOLYGON (((127.06082 37.14921, 127.06082 ..."
3737,다사622056,4,"MULTIPOLYGON (((127.07434 37.14836, 127.07433 ..."
4155,다사632067,3,"MULTIPOLYGON (((127.08555 37.15831, 127.08554 ..."
3850,다사625043,5,"MULTIPOLYGON (((127.07778 37.13665, 127.07778 ..."
3849,다사625042,5,"MULTIPOLYGON (((127.07779 37.13575, 127.07778 ..."
2710,다사607089,3,"MULTIPOLYGON (((127.05728 37.17804, 127.05727 ..."
1752,다사594107,3,"MULTIPOLYGON (((127.04253 37.19421, 127.04253 ..."


In [60]:
acc2 = target[target['accident_cnt'] >= 2 ] # 2번이상 교통사고 일어난 곳 ( 즉, 20곳)

In [61]:
# 사고 Top 20 격자 시각화
center = [37.163619, 127.049883]
m = folium.Map(location=center, tiles='openstreetmap', zoom_start=12.8) #'cartodbpositron'
folium.GeoJson(acc2['geometry']).add_to(m)
m

In [62]:
# 마우스 호버링 시 좌표 표시되도록
MousePosition().add_to(m)

In [63]:
# 보호구역에서 필요한 column만 가져오기
protect_spot = 보호구역[['시설종류','보호구역_경도','보호구역_위도']]

In [64]:
#center = [37.163619, 127.049883]
#m = folium.Map(location=center, tiles='openstreetmap', zoom_start=13, fill_color='BuPu') #'cartodbpositron'
for i in protect_spot.index:
    folium.Marker(
        location = 보호구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 300,
        tooltip = 보호구역.loc[i,'시설명']
    ).add_to(m)

for i in protect_spot.index:
    folium.Circle(
        location = 보호구역.loc[i, ['보호구역_위도', '보호구역_경도']],
        radius = 300,
        width = 0.5,
        color='green',
        tooltip = 보호구역.loc[i,'시설종류']
    ).add_to(m)        
m

In [65]:
# 중심 위경도
target[['centroid']] = target.geometry.centroid

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [67]:
# 위/경도 column 추가
target['lon'] = target.centroid.x
target['lat'] = target.centroid.y

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports unt

In [68]:
target.head()

,gid,accident_cnt,geometry,centroid,lon,lat
3681,다사621048,10,"MULTIPOLYGON (((127.07325 37.14114, 127.07325 ...",POINT (127.07381 37.14159),127.073814,37.141594
3098,다사612101,6,"MULTIPOLYGON (((127.06285 37.18888, 127.06284 ...",POINT (127.06341 37.18933),127.063406,37.189331
3863,다사625056,6,"MULTIPOLYGON (((127.07772 37.14837, 127.07771 ...",POINT (127.07828 37.14882),127.078278,37.148821
2902,다사610057,5,"MULTIPOLYGON (((127.06082 37.14921, 127.06082 ...",POINT (127.06138 37.14966),127.061382,37.149661
3737,다사622056,4,"MULTIPOLYGON (((127.07434 37.14836, 127.07433 ...",POINT (127.07490 37.14881),127.074900,37.148809


In [69]:
target

,gid,accident_cnt,geometry,centroid,lon,lat
3681,다사621048,10,"MULTIPOLYGON (((127.07325 37.14114, 127.07325 ...",POINT (127.07381 37.14159),127.073814,37.141594
3098,다사612101,6,"MULTIPOLYGON (((127.06285 37.18888, 127.06284 ...",POINT (127.06341 37.18933),127.063406,37.189331
3863,다사625056,6,"MULTIPOLYGON (((127.07772 37.14837, 127.07771 ...",POINT (127.07828 37.14882),127.078278,37.148821
2902,다사610057,5,"MULTIPOLYGON (((127.06082 37.14921, 127.06082 ...",POINT (127.06138 37.14966),127.061382,37.149661
3737,다사622056,4,"MULTIPOLYGON (((127.07434 37.14836, 127.07433 ...",POINT (127.07490 37.14881),127.074900,37.148809
4155,다사632067,3,"MULTIPOLYGON (((127.08555 37.15831, 127.08554 ...",POINT (127.08611 37.15876),127.086106,37.158764
3850,다사625043,5,"MULTIPOLYGON (((127.07778 37.13665, 127.07778 ...",POINT (127.07834 37.13710),127.078343,37.137103
3849,다사625042,5,"MULTIPOLYGON (((127.07779 37.13575, 127.07778 ...",POINT (127.07835 37.13620),127.078348,37.136202
2710,다사607089,3,"MULTIPOLYGON (((127.05728 37.17804, 127.05727 ...",POINT (127.05784 37.17849),127.057836,37.178493
1752,다사594107,3,"MULTIPOLYGON (((127.04253 37.19421, 127.04253 ...",POINT (127.04309 37.19466),127.043093,37.194663


In [99]:
center = [37.163619, 127.049883]
m = folium.Map(location=center, tiles='openstreetmap', zoom_start=12.8) #'cartodbpositron'
folium.GeoJson(acc2['geometry'].iloc[11]).add_to(m) # iloc값 바꿔가면서 명명했음 (구글 지도도 함께 참고)
m
# iloc[0] : 원동주공 아파트 앞 경기대로
# iloc[1] : 교육지원청 사거리
# iloc[2] : 오산시청앞 삼거리
# iloc[3] : 남촌오거리
# iloc[4] : 오산비뇨기과 의원 부근 성호대로
# iloc[5] : 경기동로 삼거리???
# iloc[6] : 말머리로 / 동부대로 교차사거리 (TH 모터스 부근)
# iloc[7] : 고현 아이파크 부근 남부대로 사거리
# iloc[8] : 예다원 모텔 부근 경기대로
# iloc[9]  : 세교주유소 부근 경기대로
# iloc[10] : 원동 프루지오 아파트 부근 사거리
# iloc[11] : 롯데리아 오산세교 DT점 부근 
# iloc[12] : 오산역 환승센터 부근
# iloc[13] : 오산대학교 앞 궐동로
# iloc[14] : 오산시청 부근 성호대로
# iloc[15] : 현대블루핸즈 가수점 부근 삼거리
# iloc[16] : 세마동 주민센터 부근
# iloc[17] : 지석묘진 일대
# iloc[18] : 한일아파트 부근 경기대로
# iloc[19] : 잔다리마을 2단지 일대 수목원로
